In [ ]:
import os
import numpy as np
import cv2
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

# Helper function to load and preprocess images
def load_images_from_directory(directory, label, image_size=(32, 32)):
    data = []
    labels = []
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if os.path.isfile(file_path):
            image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, image_size)
            data.append(image.flatten())
            labels.append(label)
    return np.array(data), np.array(labels)

# Load and preprocess data
def preprocess_data():
    # Load training data
    train_cats, train_cats_labels = load_images_from_directory('train/cats', 0)
    train_dogs, train_dogs_labels = load_images_from_directory('train/dogs', 1)
    
    X_train = np.vstack((train_cats, train_dogs))
    y_train = np.hstack((train_cats_labels, train_dogs_labels))
    
    # Load test data
    test_cats, test_cats_labels = load_images_from_directory('test/cats', 0)
    test_dogs, test_dogs_labels = load_images_from_directory('test/dogs', 1)
    
    X_test = np.vstack((test_cats, test_dogs))
    y_test = np.hstack((test_cats_labels, test_dogs_labels))
    
    # Shuffle and scale the data
    X_train, y_train = shuffle(X_train, y_train, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, y_train, X_test, y_test

# Load data
X_train, y_train, X_test, y_test = preprocess_data()


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
grid = GridSearchCV(SVC(max_iter=5000), param_grid, cv=3, verbose=3)
grid.fit(X_train, y_train)

print(f"Best Parameters: {grid.best_params_}")
best_model = grid.best_estimator_


In [ ]:
# Train SVM
print("Training optimized SVM model...")
svm = SVC(kernel='rbf', max_iter=50000, verbose=True,gamma='auto',C=1) 
svm.fit(X_train, y_train)

In [ ]:
# Evaluate on test data
y_test_pred = svm.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")